#### Q1. Getting the embeddings model

In [1]:
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

/Users/zaahirdawood/Documents/llm-zoomcamp/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/zaahirdawood/Documents/llm-zoomcamp/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
user_question = "I just discovered the course. Can I still join it?"

In [3]:
v = embedding_model.encode(user_question)

round(v[0],3)


np.float32(0.078)

In [4]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [6]:
documents = [doc for doc in documents if doc['course'] in "machine-learning-zoomcamp"]

#### Q2. Creating the embeddings

Create a list embeddings

Iterate over each document

qa_text = f'{question} {text}'

compute the embedding for qa_text, append to embeddings

At the end, let X = np.array(embeddings) (import numpy as np)

What's the shape of X? (X.shape). Include the parantheses.



In [7]:
import numpy as np 
from tqdm.auto import tqdm

embeddings = []

for doc in tqdm(documents):
    qa_text = f"{doc['question']} {doc['text']}"
    vector = embedding_model.encode(qa_text)
    embeddings.append(vector)

X = np.array(embeddings)

100%|██████████| 375/375 [00:46<00:00,  7.98it/s]


In [8]:
X.shape

(375, 768)

#### Q3. Search

In [9]:
scores = X.dot(v)

round(max(scores),3)

np.float32(0.651)

#### Q4. Hit-rate for our search engine

In [10]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')


In [11]:
es_client.info()

ObjectApiResponse({'name': 'f82fbccab2a3', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'pAGy21jQTGiyE6hPAiVFnQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [12]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

In [13]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [14]:
relevance_total = []

for question in tqdm(ground_truth):
    doc_id = question['document']
    v_doc_question = embedding_model.encode(question['question'])
    results = search_engine.search(v_doc_question, num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

100%|██████████| 1830/1830 [00:29<00:00, 63.05it/s]


In [42]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [16]:
hit_rate(relevance_total)

0.9398907103825137

#### Q5. Indexing with Elasticsearch

In [17]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [18]:
documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

In [19]:
for doc in tqdm(documents):
    
    question = doc['question']
    text = doc['text']
    question_text = f"{doc['question']} {doc['text']}"
    
    doc['question_vector'] = embedding_model.encode(question)
    doc['text_vector'] = embedding_model.encode(text)
    doc['question_text_vector'] = embedding_model.encode(question_text)
    
    es_client.index(index="course-questions", document=doc)
    
    

100%|██████████| 375/375 [00:47<00:00,  7.87it/s]


In [20]:
def elastic_search(field, vector, num_results):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": num_results,
        "num_candidates": 10000
    }

    search_query = {
        "knn": knn,
        "_source": ["id", "text", "section", "question", "course"]  
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )

    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append({
            "text": hit['_source']['text'],
            "section": hit['_source'].get('section', 'Unknown'),  
            "question": hit['_source'].get('question', 'Unknown'),
            "course": hit['_source'].get('course', 'Unknown'),
            "id": hit['_source'].get('id')
        })
    
    return result_docs

In [21]:
query = "I just discovered the course. Can I still join it?"

query_embeddings = embedding_model.encode(query)

In [22]:
elastic_search("question_vector",query_embeddings, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

In [23]:
[doc for doc in documents if doc["id"] == "ee58a693"]

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693',
  'question_vector': array([ 0.07717118, -0.04749377,  0.02866323, -0.01141068,  0.08245005,
         -0.04042275, -0.02613377,  0.04122073, -0.04840765,  0.01509398,
         -0.0014967 , -0.01334827,  0.0461828 ,  0.02318393,  0.04547327,
         -0.00809984,  0.07718321, -0.03334848, -0.04180019, -0.02304639,
         -0.01866887,  0.00298914, -0.0063176 ,  0.039316  , -0.0228994 ,
          0.07724466,  0.06296353,  0.037800

#### Q6. Hit-rate for Elasticsearch

In [24]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [43]:
def question_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = embedding_model.encode(question)

    return elastic_search('question_vector', v_q, 15)

In [45]:
query = "I just discovered the course. Can I still join it?"
v_q = embedding_model.encode(query)

In [46]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),

    }

In [47]:
evaluate(ground_truth, question_vector_knn)

100%|██████████| 1830/1830 [00:45<00:00, 40.10it/s]


{'hit_rate': 0.8508196721311475}